In [1]:
import pandas as pd
import numpy as np
import requests
import optuna

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

from implicit.bpr import BayesianPersonalizedRanking

from implicit.lmf import LogisticMatrixFactorization


import warnings

warnings.filterwarnings(action='ignore', category=UserWarning)

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
from implicit.gpu import HAS_CUDA

print('HAS_CUDA:', HAS_CUDA)

HAS_CUDA: False


In [3]:
import os

os.environ['OPENBLAS_NUM_THREADS'] = '1'

In [4]:
interactions = pd.read_csv('../datasets/kion/interactions.csv')
users = pd.read_csv('../datasets/kion/users.csv')
items = pd.read_csv('../datasets/kion/items.csv')

# Обработка данных

In [5]:
Columns.Datetime = 'last_watch_dt'

In [6]:
interactions.drop(
    interactions[interactions[Columns.Datetime].str.len() != 10].index,
    inplace=True,
)
interactions[Columns.Datetime] = pd.to_datetime(
    interactions[Columns.Datetime], format='%Y-%m-%d'
)
max_date = interactions[Columns.Datetime].max()
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [7]:
# Разделяем на train и test
train = interactions[
    interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)
].copy()
test = interactions[
    interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)
].copy()

In [8]:
train.drop(train.query('total_dur < 300').index, inplace=True)
cold_users = set(test[Columns.User]) - set(train[Columns.User])
len(cold_users)

72930

In [9]:
# Отбрасываем холодных пользователей
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Подготовка фич

## User features

In [10]:
users.fillna('Unknown', inplace=True)
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [11]:
user_features_frames = []
for feature in ['sex', 'age', 'income']:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ['id', 'value']
    feature_frame['feature'] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


## Item features

In [12]:
items.fillna('Unknown', inplace=True)
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

C:\Users\PC\AppData\Local\Temp\ipykernel_3464\2719428295.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  items.fillna('Unknown', inplace=True)


In [13]:
items['genre'] = (
    items['genres']
    .str.lower()
    .str.replace(', ', ',', regex=False)
    .str.split(',')
)
genre_feature = items[['item_id', 'genre']].explode('genre')
genre_feature.columns = ['id', 'value']
genre_feature['feature'] = 'genre'
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [14]:
content_feature = items.reindex(columns=[Columns.Item, 'content_type'])
content_feature.columns = ['id', 'value']
content_feature['feature'] = 'content_type'

In [15]:
countries_feature = items.reindex(columns=[Columns.Item, 'countries'])
countries_feature.columns = ['id', 'value']
countries_feature['feature'] = 'countries'

In [16]:
item_features = pd.concat((genre_feature, content_feature, countries_feature))

In [17]:
metrics_name = {
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [18]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=['sex', 'age', 'income'],
    item_features_df=item_features,
    cat_item_features=['genre', 'content_type', 'countries'],
)

TEST_USERS = test[Columns.User].unique()

# Подбор гиперпараметров

## Подбор гиперпараметров ImplicitALS

In [19]:
def ials_objective(trial):
    factors = trial.suggest_categorical('factors', [4, 8, 16, 32, 64])
    regularization = trial.suggest_float('regularization', 0.001, 0.1, log=True)
    iterations = trial.suggest_categorical('iterations', [1, 3, 5, 10, 15])

    model = ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            iterations=iterations,
            random_state=42,
            num_threads=4,
            use_gpu=False,
        ),
        fit_features_together=True,
    )

    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=10,
        filter_viewed=True,
    )

    map10 = calc_metrics(metrics, recos, test, train)['MAP@10']
    return map10

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(ials_objective, n_trials=20)

In [ ]:
print(f'Best MAP@10 value: {study.best_value}')
print(f'Best parameters: {study.best_params}')

In [ ]:
# Best MAP@10 value: 0.075254069127234
# Best parameters: {'factors': 32, 'regularization': 0.01883534498756549, 'iterations': 5}

# Лучшая модель

In [20]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=['sex', 'age', 'income'],
    item_features_df=item_features,
    cat_item_features=['genre', 'content_type', 'countries'],
)

TEST_USERS = test[Columns.User].unique()

In [21]:
model = ImplicitALSWrapperModel(
    model=AlternatingLeastSquares(
        factors=32,
        regularization=0.01883534498756549,
        iterations=5,
        random_state=42,
        num_threads=4,
        use_gpu=False,
    ),
    fit_features_together=True,
)

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


# Искусственные пользователи

Егор - любитель фильмов Квентина Тарантино

In [22]:
users = pd.concat(
    [
        users,
        pd.DataFrame([
            {
                'user_id': 1100000,
                'age': 'age_18_24',
                'income': 'income_60_90',
                'sex': 'М',
                'kids_flg': 0,
            }
        ]),
    ],
    ignore_index=True,
)


In [23]:
users.iloc[-1]

user_id          1100000
age            age_18_24
income      income_60_90
sex                    М
kids_flg               0
Name: 586668, dtype: object

In [24]:
items.loc[
    items['item_id'].isin([14804, 7693, 11115, 8148, 16382, 4072, 898, 13715]),
    ['item_id', 'title', 'content_type', 'countries', 'genre'],
]

,item_id,title,content_type,countries,genre
107,14804,Криминальное чтиво,film,США,"[зарубежные, триллеры, криминал, комедии]"
3134,16382,Человек-паук,film,США,"[боевики, фантастика, приключения]"
4766,7693,Джанго освобождённый,film,США,"[боевики, драмы, вестерн, комедии]"
5806,11115,Бесславные ублюдки,film,"Германия, США","[боевики, драмы, военные, комедии]"
6992,898,Большой куш,film,"Великобритания, США","[боевики, комедии]"
10425,13715,Стекло,film,"США, Китай","[драмы, фантастика, триллеры]"
13803,8148,Бешеные псы,film,США,"[драмы, триллеры]"
15625,4072,Человек-паук 3: Враг в отражении,film,США,"[боевики, фантастика, приключения]"


In [25]:
first_items = [14804, 7693, 11115, 8148, 16382, 4072, 898, 13715]
first_num_items = len(first_items)
first_avatar = pd.DataFrame({
    'user_id': np.full(first_num_items, fill_value=1100000),
    'item_id': first_items,
    'last_watch_dt': np.full(first_num_items, fill_value='2021-05-29'),
    'total_dur': np.full(first_num_items, fill_value=np.nan),
    'watched_pct': first_num_items * [100.0],
    'weight': [3, 3, 3, 3, 3, 1, 3, 3],
})

first_avatar[Columns.Datetime] = pd.to_datetime(
    first_avatar[Columns.Datetime], format='%Y-%m-%d'
)

In [26]:
avatars = pd.concat([first_avatar])
train = pd.concat([train, avatars], ignore_index=True)

In [27]:
# Соберем датасет
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=['sex', 'age', 'income'],
    item_features_df=item_features,
    cat_item_features=['genre', 'content_type', 'countries'],
)

avatars_ids = avatars['user_id'].unique()

In [28]:
model.fit(dataset)

In [29]:
recos = model.recommend(
    users=avatars_ids,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

In [30]:
recs = recos.merge(
    items[['item_id', 'title', 'content_type', 'countries', 'genre']],
    on='item_id',
).sort_values(['user_id', 'rank'])

In [31]:
# Рекомендации для пользователя, смотревшего только сериалы
recs[recs['user_id'] == avatars_ids[0]]

,user_id,item_id,score,rank,title,content_type,countries,genre
0,1100000,15297,0.076061,1,Клиника счастья,series,Россия,"[драмы, мелодрамы]"
1,1100000,12798,0.037087,2,Игрушки для взрослых,film,"США, Китай","[боевики, популярное, триллеры, комедии]"
2,1100000,1287,0.035975,3,Терминатор: Тёмные судьбы,film,"США, Китай","[боевики, фантастика, приключения]"
3,1100000,5521,0.035578,4,Трансформеры 4: Эпоха истребления,film,"США, Китай","[боевики, фантастика, приключения]"
4,1100000,9728,0.035515,5,Гнев человеческий,film,"Великобритания, США","[боевики, триллеры]"
5,1100000,1382,0.035079,6,Удача Логана,film,"США, Китай","[боевики, криминал, комедии]"
6,1100000,5869,0.035047,7,Эверест,film,"США, Китай","[боевики, драмы, триллеры, приключения]"
7,1100000,7573,0.035018,8,Полтора шпиона,film,"США, Китай","[боевики, криминал, комедии]"
8,1100000,3673,0.034021,9,Джейсон Борн,film,"США, Китай","[боевики, триллеры]"
9,1100000,7671,0.033983,10,Джон Уик,film,"США, Китай","[боевики, триллеры]"
